# Check GWPs in table 7.SM.7

Theme Song: Low Tide<br>
Artist: Isis, Aereogramme<br>
Album: In The Fishtank 14<br>
Released: 2006

In [4]:
from fair.tools.gwp import gwp
from fair.constants import molwt
import pandas as pp
import numpy as np

from ar6.metrics.halogen_generic import halogen_analytical
from ar6.metrics.co2 import co2_analytical
from ar6.metrics.ch4 import ch4_analytical

from ar6.constants.gases import lifetimes, radeff

from fair.constants.general import M_ATMOS

## First let's get the AGWP20 and AGWP100 for CO2

In [25]:
co2_analytical(20)   # rf, agwp, delT, idelT

(1.0189786364138617e-15,
 2.433921440465611e-14,
 4.681161234177259e-16,
 8.54380577377305e-15)

In [44]:
co2_analytical(50, d=np.array([3.424102092311, 285.003477841911]), q=np.array([0.443767728883447, 0.319591049742508]))

(8.314494575471006e-16,
 5.1724670783071825e-14,
 4.2867897555143954e-16,
 2.3161964643180585e-14)

In [65]:
co2_analytical(100, d=np.array([3.424102092311, 285.003477841911]), q=np.array([0.443767728883447, 0.319591049742508]), co2=410, ch4=1866, n2o=332, a=np.array([0.2173, 0.2240, 0.2824, 0.2763]), alpha_co2=np.array([0, 394.4, 36.54, 4.304]))

(6.994622015869083e-16,
 8.944343008172272e-14,
 3.9595361834127504e-16,
 4.355995491730908e-14)

In [27]:
co2_analytical(500)

(4.791190255031502e-16,
 3.1383888228822573e-13,
 3.5889636432888816e-16,
 1.927170335076471e-13)

In [33]:
rf, agwp, delT, idelT = co2_analytical(np.arange(101))

In [50]:
### Bill's Thomas Gasser IRF to move to a new module

# GTP is the ratio of "delT" values between gas of interest and CO2

from fair.constants import molwt

def ccycle_gasser(H, delT):
    dts = H[1]
    rf_co2, agwp_co2, delT_co2, idelT_co2 = co2_analytical(H)
    
    delT_cc = H*0.
    agwp_cc = H*0.
    rf_cc = H*0.
    F_CO2 = H*0.
    a = np.array([0.6368, 0.3322, 0.0310])  # Gasser et al. 2017
    alpha = np.array([2.376, 30.14, 490.1])

    gamma = 3.015*1E12  # kgCO2/yr/K  Gasser et al. 2017
    r_f = H*0.
    r_f[0] = np.sum(a)/dts
    for i in np.arange(0, 3):
        r_f = r_f-(a[i]/alpha[i])*np.exp(-H/alpha[i])

    for j in np.arange(H.size):
        for i in np.arange(j+1):
            F_CO2[j] = F_CO2[j]+delT[i]*gamma*r_f[j-i]*dts
    for j in np.arange(H.size):
        for i in np.arange(j+1):
            rf_cc[j] = rf_cc[j]+F_CO2[i]*rf_co2[j-i]*dts * \
                (molwt.CO2/12)
            agwp_cc[j] = agwp_cc[j]+F_CO2[i]*agwp_co2[j-i]*dts * \
                (molwt.CO2/12)
            delT_cc[j] = delT_cc[j]+F_CO2[i]*delT_co2[j-i]*dts * \
                (molwt.CO2/12)
    return rf_cc, agwp_cc, delT_cc


In [51]:
ccycle_gasser(np.arange(101), delT)

(array([ 0.00000000e+00,  2.02432091e-18,  4.86098206e-18,  7.84992822e-18,
         1.06577487e-17,  1.31389858e-17,  1.52519625e-17,  1.70086535e-17,
         1.84455955e-17,  1.96076245e-17,  2.05393016e-17,  2.12808358e-17,
         2.18665516e-17,  2.23247222e-17,  2.26780753e-17,  2.29445728e-17,
         2.31382474e-17,  2.32699868e-17,  2.33482168e-17,  2.33794699e-17,
         2.33688437e-17,  2.33203614e-17,  2.32372497e-17,  2.31221498e-17,
         2.29772754e-17,  2.28045287e-17,  2.26055846e-17,  2.23819521e-17,
         2.21350164e-17,  2.18660693e-17,  2.15763293e-17,  2.12669559e-17,
         2.09390579e-17,  2.05937000e-17,  2.02319060e-17,  1.98546612e-17,
         1.94629138e-17,  1.90575756e-17,  1.86395229e-17,  1.82095965e-17,
         1.77686025e-17,  1.73173125e-17,  1.68564645e-17,  1.63867632e-17,
         1.59088806e-17,  1.54234575e-17,  1.49311034e-17,  1.44323981e-17,
         1.39278924e-17,  1.34181090e-17,  1.29035437e-17,  1.23846663e-17,
         1.1

In [46]:
-3.49675758e-18 + 3.960979480442774e-16

3.9260119046427737e-16

## First, we'll do the halogens

In [3]:
lifetimes['CFC-11'], radeff['CFC-11'], molwt.CFC11
halogen_analytical(100, lifetimes['CFC-11'], radeff['CFC-11'], molwt.CFC11, 0.13)

(1.7594733595013092e-12,
 5.344978747321651e-10,
 1.3486608050322899e-12,
 2.571559526658994e-10)

In [8]:
halogen_analytical(100, lifetimes['CFC-11'], 0.927*radeff['CFC-11'], molwt.CFC11, 0.13)

(1.6310318042577135e-12,
 4.954795298767171e-10,
 1.2502085662649327e-12,
 2.383835681212887e-10)

In [12]:
halogen_analytical(100, 52, 0.25941, 137.36, 0.12)

(1.7440297573591152e-12,
 5.298063728809798e-10,
 1.3368230691636476e-12,
 2.5489879190809475e-10)

In [2]:
# Raw metrics file from Bill Collins
df = pd.read_csv('../data_input/7sm/metrics_supplement.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data_input/7sm/metrics_supplement.csv'

In [ ]:
df

In [ ]:
lt = df.loc[df['Acronym']=='CFC-11', 'Lifetime (yr)'].values[0]
re = df.loc[df['Acronym']=='CFC-11', 'Radiative efficiency (W m-2 ppb-1)'].values[0]

In [ ]:
re

In [ ]:
gwp(100, lt, re, molwt.CFC11)